In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import glob
import lib
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Load Data

Trading on [Xetra](https://www.xetra.com/xetra-en/trading/Trading-calendar-and-trading-hours-22048) is from Mondays to Fridays from 9 until 17:30 CET.

In [3]:
df = lib.read_date_range(start_date='2020-08-01')
df.head()

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,CalcDateTime
83,DE0006062144,1COV,COVESTRO AG O.N.,Common stock,EUR,2505008,33.02,33.140,33.02,33.13,11421.0,21.0,2020-08-03 07:00:00
152,DE000TRAT0N7,8TRA,TRATON SE INH O.N.,Common stock,EUR,4197490,16.10,16.234,16.00,16.20,3494.0,11.0,2020-08-03 07:00:00
44,DE0005093108,AAD,AMADEUS FIRE AG,Common stock,EUR,2504879,98.30,98.300,97.80,97.80,366.0,5.0,2020-08-03 07:00:00
132,LU1250154413,ADJ,ADO PROPERTIES S.A. NPV,Common stock,EUR,2506119,23.90,24.100,23.90,24.10,2702.0,11.0,2020-08-03 07:00:00
10,DE000A1EWWW0,ADS,ADIDAS AG NA O.N.,Common stock,EUR,2504471,234.80,234.900,234.00,234.30,11917.0,55.0,2020-08-03 07:00:00


In [4]:
df.shape

(1281239, 13)

In [29]:
import sys
sys.getsizeof(df)

555725406

In [10]:
lib.plot_security(df, 'BMW', 'EndPrice', 'line', 'D', True)

Anamalous volume means something? : Yes, something related to ETF

## Differences

EndPrice - StartPrice tells you have much the price moved between two periods. 

In [17]:
df.head(2)

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,CalcDateTime,PriceChange
83,DE0006062144,1COV,COVESTRO AG O.N.,Common stock,EUR,2505008,33.02,33.140,33.02,33.13,11421.0,21.0,2020-08-03 07:00:00,0.11
152,DE000TRAT0N7,8TRA,TRATON SE INH O.N.,Common stock,EUR,4197490,16.10,16.234,16.00,16.20,3494.0,11.0,2020-08-03 07:00:00,0.10


In [19]:
df.set_index('CalcDateTime', drop=True, inplace=True)

In [20]:
df['PriceChange'] = df['EndPrice'] - df['StartPrice']

In [21]:
lib.plot_security(df, 'BMW', {'PriceChange': 'sum'}, 'line', '1h', True)

In [25]:
df_ = df[df.Mnemonic == 'BMW']
df_.head()

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,PriceChange
CalcDateTime,,,,,,,,,,,,,
2020-08-03 07:00:00,DE0005190003,BMW,BAY.MOTOREN WERKE AG ST,Common stock,EUR,2504900,54.70,54.85,54.58,54.68,30003.0,57.0,-0.02
2020-08-03 07:01:00,DE0005190003,BMW,BAY.MOTOREN WERKE AG ST,Common stock,EUR,2504900,54.73,54.77,54.45,54.45,2543.0,22.0,-0.28
2020-08-03 07:02:00,DE0005190003,BMW,BAY.MOTOREN WERKE AG ST,Common stock,EUR,2504900,54.48,54.64,54.47,54.64,3233.0,21.0,0.16
2020-08-03 07:03:00,DE0005190003,BMW,BAY.MOTOREN WERKE AG ST,Common stock,EUR,2504900,54.58,54.59,54.42,54.47,7861.0,30.0,-0.11
2020-08-03 07:04:00,DE0005190003,BMW,BAY.MOTOREN WERKE AG ST,Common stock,EUR,2504900,54.50,54.70,54.50,54.64,3341.0,28.0,0.14


In [27]:
df.between_time('08:00', '16:00').shape

(1092436, 13)

In [28]:
df.shape

(1281239, 13)

Is there a pattern in days where the market opens much higher/lower wrt to last closing price? 

In [15]:
ohlcv_dict = {'StartPrice':'first', 'MaxPrice':'max', 'MinPrice':'min', 'EndPrice': 'last', 
          'TradedVolume': 'sum', 'NumberOfTrades': 'sum'}

In [24]:
daily_df = df_.resample('D').apply(ohlcv_dict)
daily_df.dropna(inplace=True)
daily_df.head()

,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
CalcDateTime,,,,,,
2020-08-03,54.70,56.61,54.42,56.21,1812786.0,10217.0
2020-08-04,57.09,58.14,56.96,58.14,1873398.0,8600.0
2020-08-05,56.50,56.93,55.32,56.13,3067112.0,12735.0
2020-08-06,55.98,56.23,54.75,55.23,2453738.0,11765.0
2020-08-07,55.16,55.40,54.30,54.90,1984567.0,8735.0
